In [1]:
# pip install bitsandbytes accelerate flash-attn

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
import torch
import bz2
import pickle
import importlib
from tqdm import tqdm


import time
from IPython.display import JSON
import sys
sys.settrace(None)
import pdb

import copy

import psycopg
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer, StoppingCriteria, StoppingCriteriaList, DynamicCache, OffloadedCache #, CodeGenTokenizer
from transformers.generation.logits_process import LogitsProcessorList

In [4]:
assert torch.cuda.is_available()

In [5]:
### Remind to remove '.py'
# index_module = importlib.import_module('phi4_index')
#index_module = importlib.import_module('http_index')
#index_module = importlib.import_module('qwen25_index')
index_module = importlib.import_module('gemma3_index')
#index_module = importlib.import_module('llama_index')
index_config = getattr(index_module, 'index_config')

# model_module = importlib.import_module('phi4_model')
#model_module = importlib.import_module('qwen25_3B_model')
model_module = importlib.import_module('gemma3_1B_model')
#model_module = importlib.import_module('llama_1B_model')
model_config = getattr(model_module, 'model_config')

Loading google/gemma-3-1b-it


In [6]:
streamer = TextStreamer(model_config.tokenizer)

In [7]:
from ctrie import ConstrainedLogitsProcessor, ConstrainedStateList, \
                    ConstrainedState, DictIndex

In [8]:
question = '''Which city is the capital of the country where the Eiffel Tower is?'''
question = '''Who was the quarterback of the team that won Super Bowl 50?'''
question = '''Who is the lead character in the Uncharted series?'''
question = '''Who played Gandalf in the Lord of the Rings movies?'''
question = '''What was Nirvana's last studio album called?''' # TODO test with beamsearch 3. with beam search if I have only 1 allowed token the other beams could generate something illegal and than crash
# TODO understand the crash is due to beam search and abandon those illegal paths. or try to explore always different triples with beam search???


question = '''How many members were there in the original NSYNC music group?'''

question = 'Who influenced Nirvana band?'

question = 'Is Johnny Depp or Brad Pitt older?'
question = 'Who is the mother of the director of film Polish-Russian War (Film)?' # Małgorzata Braunek
question = 'Which film came out first, Blind Shaft or The Mask Of Fu Manchu?' # The Mask Of Fu Manchu
question = 'When did John V, Prince Of Anhalt-Zerbst\'s father die?' # 12 June 1516

prompted_texts = [model_config.apply_prompt_template(question)]

In [9]:
print(prompted_texts[0])

<bos><start_of_turn>user
You are a helpful question answering assistant that bases its answer on facts from a knowledge base.
1) You receive an input question.
2) You reason on the path you need to follow to reach the answer starting from the information in the question.
3) You explicitly provide relevant facts, one per line starting with "Fact:" and finish with ".".
4) You fully trust only the facts that starts with "Fact:".
5) You explain your reasoning process and provide a long answer with your motivations based on the facts.
6) You provide a short concise answer.

Sometimes it could happen that there are no relevant facts in the knowledge base.
In these cases you either:
a) provide your own answer clearly stating that there were no supporting facts to your answer;
b) in case you don't know the answer or you are not confident enough, you just answer "I don't know".

Remind that you can use <description> and <short description> for acquiring general information about relevant entiti

In [10]:
"""prompted_texts = [
    '''
<|im_start|>system
You are a helpful question answering assistant that bases its answer on facts from a knowledge base.
1) You receive an input question.
2) You reason on the path you need to follow to reach the answer starting from the information in the question.
3) You explicitly provide relevant facts, one per line starting with "Fact:".
4) You explain your reasoning process and provide a long answer with your motivations based on the facts.
5) You provide a short concise answer.

Sometimes it could happen that there are no relevant facts in the knowledge base.
In these cases you either:
a) provide your own answer clearly stating that there were no supporting facts to your answer;
b) in case you don't know the answer or you are not confident enough, you just answer "I don't know".
<|im_end|>
<|im_start|>user
Which mountain is taller between Mont Blanc and Mount Rainier?
<|im_end|>
<|im_start|>assistant
Reasoning: I need to provide the height of Mont Blanc and the height of Mount Rainier, then I need to compare the two heights and the final answer will be the taller mountain.
Fact: <Mont Blanc> <elevation above sea level> <4,807.02±0.5 meters> .
Now I need the height of Mount Rainier.
Fact: <Mount Rainier> <elevation above sea level> <4,389 meters> .
Long answer: Mont Blanc is 4,807 meters tall, while Mount Rainier is 4,389 meters, so Mont Blanc is taller than Mount Rainier.

Answer: Mont Blanc.
<|im_end|>
<|im_start|>user
What is the fourth book in the Twilight series?<|im_end|>
<|im_start|>assistant
Reasoning: To answer the question about the fourth book in the Twilight series, I need to know the titles of the books in the series. I will search for the titles of the books in the Twilight series and identify the fourth one.
''']
prompted_texts = ['''<|im_start|>system
You are a helpful question answering assistant that bases its answer on facts from a knowledge base.
1) You receive an input question.
2) You reason on the path you need to follow to reach the answer starting from the information in the question.
3) You explicitly provide relevant facts, one per line starting with \"Fact:\".
4) You explain your reasoning process and provide a long answer with your motivations based on the facts.
5) You provide a short concise answer.

Sometimes it could happen that there are no relevant facts in the knowledge base.
In these cases you either:
a) provide your own answer clearly stating that there were no supporting facts to your answer;
b) in case you don't know the answer or you are not confident enough, you just answer \"I don't know\".
<|im_end|>
<|im_start|>user
Which mountain is taller between Mont Blanc and Mount Rainier?
<|im_end|>
<|im_start|>assistant
Reasoning: I need to provide the height of Mont Blanc and the height of Mount Rainier, then I need to compare the two heights and the final answer will be the taller mountain.
Fact: <Mont Blanc> <elevation above sea level> <4,807.02\u00b10.5 meters> .
Now I need the height of Mount Rainier.
Fact: <Mount Rainier> <elevation above sea level> <4,389 meters> .
Long answer: Mont Blanc is 4,807 meters tall, while Mount Rainier is 4,389 meters, so Mont Blanc is taller than Mount Rainier.

Answer: Mont Blanc.
<|im_end|>
<|im_start|>user
Is Johnny Depp or Brad Pitt older?<|im_end|>
<|im_start|>assistant




''']"""

'prompted_texts = [\n    \'\'\'\n<|im_start|>system\nYou are a helpful question answering assistant that bases its answer on facts from a knowledge base.\n1) You receive an input question.\n2) You reason on the path you need to follow to reach the answer starting from the information in the question.\n3) You explicitly provide relevant facts, one per line starting with "Fact:".\n4) You explain your reasoning process and provide a long answer with your motivations based on the facts.\n5) You provide a short concise answer.\n\nSometimes it could happen that there are no relevant facts in the knowledge base.\nIn these cases you either:\na) provide your own answer clearly stating that there were no supporting facts to your answer;\nb) in case you don\'t know the answer or you are not confident enough, you just answer "I don\'t know".\n<|im_end|>\n<|im_start|>user\nWhich mountain is taller between Mont Blanc and Mount Rainier?\n<|im_end|>\n<|im_start|>assistant\nReasoning: I need to provide

In [11]:
inputs = model_config.tokenizer(prompted_texts, return_tensors='pt', padding=True, padding_side='right')
inputs = inputs.to(model_config.model.device)

In [12]:
model_config.model.device

device(type='cuda', index=0)

In [13]:
torch.cuda.empty_cache()

In [14]:
num_beams = 1

auto_streamer = streamer if num_beams == 1 else None

states = ConstrainedStateList(
    [ConstrainedState(
                begin_pattern = model_config.switch_pattern,
                end_pattern = model_config.newline_token,
                cache_index = DictIndex(end_of_triple=index_config.index.end_of_triple),
                subtree_cache = DictIndex(end_of_triple=index_config.index.end_of_triple),
                oneleaf_cache = DictIndex(end_of_triple=index_config.index.end_of_triple)
            ) for _ in range(num_beams)],
            num_beams=num_beams,
            batch_size = 1,
            pad_token_id = model_config.tokenizer.eos_token_id)

constrained_processor = ConstrainedLogitsProcessor(
    index=index_config.index,
    end_token=model_config.newline_token, states=states, tokenizer=model_config.tokenizer)
logits_processor_list = LogitsProcessorList([
    constrained_processor
])

model_config.model.eval()
start = time.time()

with torch.no_grad():

    #pdb.set_trace()
    out = model_config.model.generate(
        **inputs,
        logits_processor=logits_processor_list,
        max_new_tokens=800,
        streamer = auto_streamer,
        do_sample = False,
        temperature = 0.0,
        top_k=None,
        num_beams=num_beams,
        #num_beam_groups = num_beams // 2,
        ## TODO group beam search works differently with logitsprocessors: input ids shape is group shape = num_beams / num_beam_groups
        #diversity_penalty=0.5,
        #num_return_sequences=1,
        num_return_sequences=num_beams,
        #no_repeat_ngram_size=1,
        #remove_invalid_values=True,
        #stopping_criteria=stopping_criteria,
        use_cache=True,
        #past_key_values=past_key_values,
        kwargs = {'constrained_state': states}, # passing state
    )

print('Elapsed', time.time() - start)

<bos><bos><start_of_turn>user
You are a helpful question answering assistant that bases its answer on facts from a knowledge base.
1) You receive an input question.
2) You reason on the path you need to follow to reach the answer starting from the information in the question.
3) You explicitly provide relevant facts, one per line starting with "Fact:" and finish with ".".
4) You fully trust only the facts that starts with "Fact:".
5) You explain your reasoning process and provide a long answer with your motivations based on the facts.
6) You provide a short concise answer.

Sometimes it could happen that there are no relevant facts in the knowledge base.
In these cases you either:
a) provide your own answer clearly stating that there were no supporting facts to your answer;
b) in case you don't know the answer or you are not confident enough, you just answer "I don't know".

Remind that you can use <description> and <short description> for acquiring general information about relevant e

/opt/conda/envs/gemma3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Reasoning: I need to find the date of John V’s father’s death.
Fact: <John V> <short description> <Topics referred to by the same term> .
Fact: <John V> <description> <Wikimedia disambiguation page> .
Fact: <John V> <instance of> <Wikimedia human name disambiguation page (Wikimedia disambiguation page for humans with the same name)> .
Fact: <John V, Prince of Anhalt-Zerbst> <date of death> <1551-02-04T00:00:00Z> .
Answer: 1551.<end_of_turn>
Elapsed 16.593607664108276


In [15]:
torch.cuda.empty_cache()

In [16]:
_from = len(inputs.input_ids[0]) # 0
for i in range(out.shape[0]):
    print('-'*30, sum(out[i][_from:]), len(out[i][_from:]))
    print(model_config.tokenizer.decode(out[i][_from:]))

------------------------------ tensor(2662662, device='cuda:0') 333
Reasoning: To determine which film came out first between "Blind Shaft" and "The Mask of Fu Manchu", I need to find the release years of both films. If I find the release year for each, I can compare them to identify the earlier one.
Fact: <Blind Shaft> <original language of film or TV show> <Standard Chinese> .
Fact: <Blind Shaft> <country of origin> <China> .
Fact: <Blind Shaft> <genre> <Drama (film and television)> .
Fact: <The Mask of Fu Manchu> <original language of film or TV show> <English language> .
Fact: <The Mask of Fu Manchu> <country of origin> <United States> .
Fact: <The Mask of Fu Manchu> <genre> <Science fiction film> .
Now I need to find the release years for both films.
Fact: <Blind Shaft> <publication date> <2005-11-10T00:00:00Z> .
Fact: <The Mask of Fu Manchu> <publication date> <1932-01-01T00:00:00Z> .
Long answer: "The Mask of Fu Manchu" was released on January 1, 1932, whereas "Blind Shaft" was 

In [17]:
for i, triple in enumerate(states[0].generated_triples):
    print(i, model_config.tokenizer.decode(triple)[:-1], triple, end='\n')

0  <Blind Shaft> <original language of film or TV show> <Standard Chinese> . [366, 4923, 484, 95759, 29, 366, 9889, 4128, 315, 4531, 476, 5883, 1473, 29, 366, 19781, 8453, 29, 659, 198]
1  <Blind Shaft> <country of origin> <China> . [366, 4923, 484, 95759, 29, 366, 11141, 315, 6238, 29, 366, 22282, 29, 659, 198]
2  <Blind Shaft> <genre> <Drama (film and television)> . [366, 4923, 484, 95759, 29, 366, 33613, 29, 366, 35, 30373, 320, 30155, 323, 12425, 16018, 659, 198]
3  <The Mask of Fu Manchu> <original language of film or TV show> <English language> . [366, 785, 19924, 315, 29253, 2363, 65752, 29, 366, 9889, 4128, 315, 4531, 476, 5883, 1473, 29, 366, 22574, 4128, 29, 659, 198]
4  <The Mask of Fu Manchu> <country of origin> <United States> . [366, 785, 19924, 315, 29253, 2363, 65752, 29, 366, 11141, 315, 6238, 29, 366, 22372, 4180, 29, 659, 198]
5  <The Mask of Fu Manchu> <genre> <Science fiction film> . [366, 785, 19924, 315, 29253, 2363, 65752, 29, 366, 33613, 29, 366, 35400, 16989, 